In [3]:
# ! pip install dash
# ! pip install dash-bootstrap-components
# ! pip install plotly

     |████████████████████████████████| 71kB 4.1MB/s eta 0:00:011
     |████████████████████████████████| 1.2MB 5.7MB/s eta 0:00:01
     |████████████████████████████████| 3.4MB 12.4MB/s eta 0:00:01
     |████████████████████████████████| 194kB 16.5MB/s eta 0:00:01
     |████████████████████████████████| 1.8MB 15.7MB/s eta 0:00:01
     |████████████████████████████████| 798kB 10.0MB/s eta 0:00:01
  Stored in directory: /Users/melissamunz/Library/Caches/pip/wheels/2e/bc/dd/b4ebecb434c90eff7c5475efc10b72aad5a899c01fa87e2647
  Stored in directory: /Users/melissamunz/Library/Caches/pip/wheels/f7/e9/e4/5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
  Stored in directory: /Users/melissamunz/Library/Caches/pip/wheels/be/9e/83/c6531c3fdd6dba27156e7196c273148d850c97bda396fec7d0
  Stored in directory: /Users/melissamunz/Library/Caches/pip/wheels/61/5c/e3/0958785022517a2e5934e7b5167a8f824f79c96a5e440bd622
  Stored in directory: /Users/melissamunz/Library/Caches/pip/wheels/22/c0/79/ef4c2b20a23

  Stored in directory: /Users/melissamunz/Library/Caches/pip/wheels/cb/1a/10/2d993250e42f29261cca29fa89338d3c09c36923d12c34faf2
Successfully built dash-bootstrap-components


In [4]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_table
import pandas as pd
import sqlite3

import plotly.graph_objs as go


In [5]:
# conn = sqlite3.connect(r"C:\Users\MTGro\Desktop\coding\wineApp\db\wine_data.sqlite")
# c = conn.cursor()
# df = pd.read_sql("select * from wine_data", conn)
# df = df[['country', 'description', 'rating', 'price','province','title','variety','winery','color']]
# df.head(1)

In [6]:
df1 = pd.read_csv('nyc_birds_full.csv')

In [7]:
df2 = df1[df1['YEAR'] > 2000]

In [8]:
df = df2[df2['COMMON NAME'] == 'Northern Cardinal']

In [9]:
df = df[['YEAR', 'OBSERVATION_COUNT', 'LATITUDE', 'LONGITUDE', 'COMMON NAME']]

In [10]:
df.head()

,YEAR,OBSERVATION_COUNT,LATITUDE,LONGITUDE,COMMON NAME
24334,2001,1,40.771548,-73.972482,Northern Cardinal
24335,2001,2,40.726320,-73.980100,Northern Cardinal
24386,2001,10,40.771548,-73.972482,Northern Cardinal
24387,2001,8,40.771548,-73.972482,Northern Cardinal
24537,2001,1,40.775709,-73.968430,Northern Cardinal


In [14]:
import plotly.graph_objects as go
import pandas as pd


data = []
layout = dict(
    title = 'All Cardinal Sightings in Manhattan<br>\
Source: <a href="http://www.econ.umn.edu/~holmes/data/WalMart/index.html">\
University of Minnesota</a>',
    # showlegend = False,
    autosize = False,
    width = 1000,
    height = 900,
    hovermode = False,
    legend = dict(
        x=0.7,
        y=-0.1,
        bgcolor="rgba(255, 255, 255, 0)",
        font = dict( size=11 ),
    )
)
years = df['YEAR'].unique()

for i in range(len(years)):
    geo_key = 'geo'+str(i+1) if i != 0 else 'geo'
    lons = list(df[ df['YEAR'] == years[i] ]['LONGITUDE'])
    lats = list(df[ df['YEAR'] == years[i] ]['LATITUDE'])
    # location data
    data.append(
        dict(
            type = 'scattergeo',
            showlegend=False,
            lon = lons,
            lat = lats,
            geo = geo_key,
            name = int(years[i]),
            marker = dict(
                color = "rgb(0, 0, 255)",
                opacity = 0.5
            )
        )
    )
    # Year markers
    data.append(
        dict(
            type = 'scattergeo',
            showlegend = False,
            lon = [-78],
            lat = [47],
            geo = geo_key,
            text = [years[i]],
            mode = 'text',
        )
    )
    layout[geo_key] = dict(
        scope = 'usa',
        showland = True,
        landcolor = 'rgb(229, 229, 229)',
        showcountries = False,
        domain = dict( x = [], y = [] ),
        subunitcolor = "rgb(255, 255, 255)",
    )


def draw_sparkline( domain, lataxis, lonaxis ):
    ''' Returns a sparkline layout object for geo coordinates  '''
    return dict(
        showland = False,
        showframe = False,
        showcountries = False,
        showcoastlines = False,
        domain = domain,
        lataxis = lataxis,
        lonaxis = lonaxis,
        bgcolor = 'rgba(255,200,200,0.0)'
    )

# Stores per year sparkline
layout['geo44'] = draw_sparkline({'x':[0.6,0.8], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 30.0]}, {'range':[0.0, 40.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df.groupby(by=['YEAR']).count()['OBSERVATION_COUNT']/1e1),
        lon = list(range(len(df.groupby(by=['YEAR']).count()['OBSERVATION_COUNT']/1e1))),
        line = dict( color = "rgb(0, 0, 255)" ),
        name = "Sightings per year<br>Since 2000",
        geo = 'geo44',
    )
)

# Cumulative sum sparkline
layout['geo45'] = draw_sparkline({'x':[0.8,1], 'y':[0,0.15]}, \
                                 {'range':[-5.0, 50.0]}, {'range':[0.0, 50.0]} )
data.append(
    dict(
        type = 'scattergeo',
        mode = 'lines',
        lat = list(df.groupby(by=['YEAR']).count().cumsum()['OBSERVATION_COUNT']/1e2),
        lon = list(range(len(df.groupby(by=['YEAR']).count()['OBSERVATION_COUNT']/1e1))),
        line = dict( color = "rgb(214, 39, 40)" ),
        name ="Cumulative sum<br>3176 stores total in 2006",
        geo = 'geo45',
    )
)

z = 0
COLS = 5
ROWS = 10
for y in reversed(range(ROWS)):
    for x in range(COLS):
        print(x)
        print(y)
        geo_key = 'geo'+str(z+1) if z != 0 else 'geo'
        print(geo_key)
        layout[geo_key]['domain']['x'] = [float(x)/float(COLS), float(x+1)/float(COLS)]
        layout[geo_key]['domain']['y'] = [float(y)/float(ROWS), float(y+1)/float(ROWS)]
        z=z+1
        if z > 18:
            break

fig = go.Figure(data=data, layout=layout)
fig.update_layout(width=800)
fig.show()

0
9
geo
1
9
geo2
2
9
geo3
3
9
geo4
4
9
geo5
0
8
geo6
1
8
geo7
2
8
geo8
3
8
geo9
4
8
geo10
0
7
geo11
1
7
geo12
2
7
geo13
3
7
geo14
4
7
geo15
0
6
geo16
1
6
geo17
2
6
geo18
3
6
geo19
0
5
geo20
0
4
geo21


KeyError: 'geo21'